In [37]:
import zss    #pip install zss
import xml.etree.ElementTree as ET
def get_children(node):
    my_children = [child for child in node]
    return my_children

    
def get_label(node):
    my_label = [child.tag for child in node]
    return my_label
def xmltree_dist(xml_1,xml_2):
    if xml_1==xml_2:
        return 0.0
    d  = zss.simple_distance(ET.parse(xml_1).getroot(),ET.parse(xml_2).getroot(),get_children,get_label)
    return d
#pass path_to_xmls
#xml_1 = '/home/ashish-j/workspace/Data_Analytics/Project/project_local_work/E0259_Project/Data/Contest1236/all_xml/29091733_62799939.xml'
#xml_2 = '/home/ashish-j/workspace/Data_Analytics/Project/project_local_work/E0259_Project/Data/Contest1236/all_xml/29087161_62785968.xml'
#d=xmltree_dist(xml_1,xml_2)
#print(d)

In [354]:
import pandas as pd

In [355]:
import os
import numpy as np

In [356]:
stats = pd.read_csv("./Data/Contest1236/standings_statistics.csv",delimiter='\t')
stats = stats.loc[stats['ProblemA_language'] == 'GNU C++17']

In [357]:
stats['file_name_problem_A'] = stats['ParticipantID'].astype(str)+"_"+stats['ProblemA_id'].astype(str)+".xml"
stats['file_name_problem_B'] = stats['ParticipantID'].astype(str)+"_"+stats['ProblemB_id'].astype(str)+".xml"
stats['file_name_problem_C'] = stats['ParticipantID'].astype(str)+"_"+stats['ProblemC_id'].astype(str)+".xml"


In [358]:
group_expert_stats = stats[:50]
group_noob_stats = stats[len(stats):len(stats)-50:-1]

In [374]:
def true_group(f):
    if f in group_expert_stats['file_name_problem_A'].tolist():
        return 0
    else:
        return 1

In [360]:
f_names = stats['file_name_problem_A'].tolist() 

In [361]:
f_names_top = group_expert_stats['file_name_problem_A'].tolist()[:10]
f_names_bottom = group_noob_stats['file_name_problem_A'].tolist()[:10]

In [530]:
#f_names_sample contains top 10 rankers and bottom 10 rankers code file names

In [531]:
f_names_sample = f_names_top+f_names_bottom

In [363]:
#computes similarity matrix
def similarity_xml_tree_dist(f_s):
    similarity = {}
    for i in range(len(f_s)):
        for j in range(len(f_s)):
            if i<=j:
                if f_s[i] in similarity.keys():
                    similarity[f_s[i]].append(xmltree_dist('./Data/Contest1236/all_xml/'+str(f_s[i]),'./Data/Contest1236/all_xml/'+str(f_s[j])))
                else:
                    similarity[f_s[i]] = [xmltree_dist('./Data/Contest1236/all_xml/'+str(f_s[i]),'./Data/Contest1236/all_xml/'+str(f_s[j]))]
            else:
                if f_s[i] in similarity.keys():
                    similarity[f_s[i]].append(similarity[f_s[j]][f_s.index(f_s[i])])
                else:
                    similarity[f_s[i]]=[similarity[f_s[j]][f_s.index(f_s[i])]]
        print(str(i+1)+" out of "+str(len(f_s))+" files done!")
    similarity_matrix = pd.DataFrame.from_dict(similarity,orient='index',columns=f_s) 
    return similarity_matrix
            

In [364]:
#sim = similarity_xml_tree_dist(f_names_sample)
#sim.to_csv("similarity_matrix_sample.csv")

In [533]:
#use this for analysis
sim = pd.read_csv("similarity_matrix_sample.csv")
sim = sim.iloc[:,1:]

In [548]:
##clustering

In [540]:
from sklearn.cluster import SpectralClustering

In [541]:
#Affinity matrix l (like in Data Analytics assignemnt but just here distance is tree edit distance)
l = copy.deepcopy(sim)
l = np.array(l)
sigma = 28
l = np.exp(-l/sigma**2)


In [542]:
clustering = SpectralClustering(n_clusters=2,
...         assign_labels="discretize",affinity='precomputed',
...         random_state=0).fit(l)

In [543]:
#sim.to_csv("sample_dist_csv.csv")

In [544]:
def accuracy(f_names,pred_labels):
    c=0
    for i in range(len(f_names)):
        if true_group(f_names[i])==pred_labels[i]:
            c+=1
    return c/len(f_names)
        

In [545]:
accuracy(f_names_sample,clustering.labels_)

0.5

In [546]:
clustering.labels_

array([1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1])

In [547]:
[true_group(f_names_sample[i]) for i in range(20)]

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]